## Topic Modeling

This notebook use Latent Dirichlet Allocation (LDA) to topic model the documents, then project them to 2 dimensions for display. 

In [2]:
# Import packages 
from os import listdir
import os
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
from nltk.tokenize import word_tokenize
import re
import gensim
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from gensim import corpora, models
from nltk.stem.porter import *
stemmer = PorterStemmer()
import numpy as np
np.random.seed(2018)
import nltk
nltk.download('wordnet')
import pandas as pd
import warnings
warnings.filterwarnings("ignore")
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)


[nltk_data] Downloading package stopwords to /Users/jshen/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /Users/jshen/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


## Preprocessing

In [2]:
# Lowercases everything 
# Removes stopwords and words with 3 or fewer chars 
# Stems and lemmatizes all words

def lemmatize_stemming(text):
    return stemmer.stem(WordNetLemmatizer().lemmatize(text, pos='v'))

def preprocess(text):
    result = []
    for token in gensim.utils.simple_preprocess(text):
        if token not in gensim.parsing.preprocessing.STOPWORDS and len(token) > 3:
            result.append(lemmatize_stemming(token))
    return result

In [3]:
f = open("./full_text/759082635-MIT.pdf.txt", "r")
print(preprocess(f.read()))

['teacher', 'self', 'efficaci', 'cape', 'town', 'approach', 'enhanc', 'qualiti', 'educ', 'yeseul', 'submit', 'depart', 'urban', 'studi', 'plan', 'partial', 'fulfil', 'requir', 'degre', 'master', 'citi', 'plan', 'massachusett', 'institut', 'technolog', 'archiv', 'massachusett', 'institut', 'cnolog', 'june', 'yeseul', 'right', 'reserv', 'author', 'grant', 'permiss', 'reproduc', 'distribut', 'publicli', 'paper', 'electron', 'copi', 'thesi', 'document', 'author', 'yeseul', 'depart', 'urban', 'studi', 'plan', 'accept', 'professor', 'ceasar', 'mcdowel', 'depart', 'urban', 'studi', 'plan', 'thesi', 'supervisor', 'professor', 'joseph', 'ferreira', 'depart', 'urban', 'studi', 'plan', 'chair', 'committe', 'certifi', 'teacher', 'self', 'efficaci', 'cape', 'town', 'approach', 'enhanc', 'qualiti', 'educ', 'yeseul', 'submit', 'depart', 'urban', 'studi', 'plan', 'partial', 'fulfil', 'requir', 'degre', 'master', 'citi', 'plan', 'abstract', 'person', 'teacher', 'self', 'efficaci', 'belief', 'abil', 'ov

In [6]:
class Documents(object):
    def __init__(self, dirname):
        self.dirname = dirname
 
    def __iter__(self):
        for fname in os.listdir(self.dirname):
            if ".pdf.txt" in fname:
                f = open(os.path.join(self.dirname, fname), "r")

                yield f.read().split("\n")
        
    def __len__(self):
        return len([x for x in os.listdir(self.dirname) if ".pdf.txt" in x])
 
documents = Documents('full_text_processed') 

In [9]:
# dictionary.save("dict.dict")
dictionary = corpora.Dictionary.load('dict.dict')

In [10]:
class BagOfWords(object):
    def __init__(self, documents, dictionary):
        self.documents = documents
        self.dict = dictionary
 
    def __iter__(self):
        for doc in self.documents: 
            yield self.dict.doc2bow(doc)
            
    def __len__(self): 
        return len(self.documents)
 
bow = BagOfWords(documents, dictionary) 

## Tuning
Let's determine how many passes and topics we need.

In [11]:
# Make an index to word dictionary.
temp = dictionary[0]  # This is only to "load" the dictionary.
id2word = dictionary.id2token

num_topics = 10
passes = 10

In [13]:
from gensim.models import CoherenceModel

# Compute Perplexity
print('\nPerplexity: ', lda_model.log_perplexity(bow))  

# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=documents, dictionary=dictionary, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)

As can be seen, perplexity has largely converged so 10 passes should be enough. Let's determine the correct number of topics to choose now. We will run the topic model with 10, 15, 20, ... 10 topics, optimizing for coherence.

In [14]:
topic_sizes = list(range(10, 100, 5))
coherences = []

for n in topic_sizes: 
    print("topic model w", n, "topics")

    lda_model = gensim.models.LdaModel(
        corpus=bow,
        id2word=id2word,
        alpha='auto',
        eta='auto',
        num_topics=n,
        passes=passes,
        eval_every=None
    )
    
    coherence_model_lda = CoherenceModel(model=lda_model, texts=documents, dictionary=dictionary, coherence='c_v')
    coherence_lda = coherence_model_lda.get_coherence()
    print('Coherence Score: ', coherence_lda)
    coherences.append(coherence_lda)
    

2023-01-09 11:53:57,688 : INFO : using autotuned alpha, starting with [0.016666668, 0.016666668, 0.016666668, 0.016666668, 0.016666668, 0.016666668, 0.016666668, 0.016666668, 0.016666668, 0.016666668, 0.016666668, 0.016666668, 0.016666668, 0.016666668, 0.016666668, 0.016666668, 0.016666668, 0.016666668, 0.016666668, 0.016666668, 0.016666668, 0.016666668, 0.016666668, 0.016666668, 0.016666668, 0.016666668, 0.016666668, 0.016666668, 0.016666668, 0.016666668, 0.016666668, 0.016666668, 0.016666668, 0.016666668, 0.016666668, 0.016666668, 0.016666668, 0.016666668, 0.016666668, 0.016666668, 0.016666668, 0.016666668, 0.016666668, 0.016666668, 0.016666668, 0.016666668, 0.016666668, 0.016666668, 0.016666668, 0.016666668, 0.016666668, 0.016666668, 0.016666668, 0.016666668, 0.016666668, 0.016666668, 0.016666668, 0.016666668, 0.016666668, 0.016666668]
2023-01-09 11:53:57,706 : INFO : using serial LDA version on this node


topic model w 60 topics


2023-01-09 11:53:58,599 : INFO : running online (multi-pass) LDA training, 60 topics, 10 passes over the supplied corpus of 58520 documents, updating model once every 2000 documents, evaluating perplexity every 0 documents, iterating 50x with a convergence threshold of 0.001000
2023-01-09 11:54:08,672 : INFO : PROGRESS: pass 0, at document #2000/58520
2023-01-09 11:54:21,099 : INFO : optimized alpha [0.029188093, 0.02875347, 0.02874849, 0.028926833, 0.029122084, 0.029351223, 0.029197399, 0.029199567, 0.0292673, 0.029506352, 0.029015116, 0.02878464, 0.029445572, 0.029285977, 0.029148057, 0.029138185, 0.029859966, 0.029339254, 0.029017894, 0.028978363, 0.029513188, 0.029356906, 0.028968718, 0.029038943, 0.028852891, 0.028748821, 0.028872281, 0.029569296, 0.02875647, 0.029251773, 0.029242072, 0.02923778, 0.029230308, 0.028766315, 0.029402684, 0.02962542, 0.029377842, 0.029418742, 0.028836466, 0.029341854, 0.02894814, 0.028916376, 0.029558474, 0.029835857, 0.029557325, 0.029382017, 0.02912

2023-01-09 11:55:32,436 : INFO : topic #41 (0.032): 0.016*"price" + 0.014*"market" + 0.009*"demand" + 0.009*"econom" + 0.009*"invest" + 0.009*"polici" + 0.008*"industri" + 0.007*"vehicl" + 0.006*"sector" + 0.006*"emiss"
2023-01-09 11:55:32,439 : INFO : topic #44 (0.033): 0.029*"user" + 0.013*"file" + 0.010*"game" + 0.007*"interfac" + 0.005*"player" + 0.005*"video" + 0.005*"server" + 0.005*"particip" + 0.005*"display" + 0.005*"softwar"
2023-01-09 11:55:32,449 : INFO : topic diff=2.678240, rho=0.500000
2023-01-09 11:55:42,545 : INFO : PROGRESS: pass 0, at document #10000/58520
2023-01-09 11:55:54,583 : INFO : optimized alpha [0.023595858, 0.026604956, 0.023100393, 0.02504971, 0.025963647, 0.030971482, 0.022566132, 0.02480298, 0.027057491, 0.028680671, 0.030594906, 0.027785532, 0.03249, 0.024249211, 0.02471272, 0.024661778, 0.023830673, 0.023697298, 0.02653525, 0.025701817, 0.023973072, 0.02914553, 0.031067358, 0.024754435, 0.025243327, 0.024321822, 0.022981739, 0.024810705, 0.02536107, 0

2023-01-09 11:57:04,550 : INFO : topic #31 (0.033): 0.028*"imag" + 0.018*"train" + 0.013*"cluster" + 0.011*"algorithm" + 0.011*"network" + 0.010*"dataset" + 0.008*"detect" + 0.007*"segment" + 0.007*"neural" + 0.007*"visual"
2023-01-09 11:57:04,553 : INFO : topic #44 (0.035): 0.040*"user" + 0.011*"interfac" + 0.010*"file" + 0.007*"display" + 0.007*"game" + 0.007*"softwar" + 0.006*"text" + 0.006*"particip" + 0.006*"visual" + 0.005*"video"
2023-01-09 11:57:04,555 : INFO : topic #12 (0.035): 0.025*"algorithm" + 0.012*"matrix" + 0.012*"theorem" + 0.011*"proof" + 0.011*"vector" + 0.010*"graph" + 0.009*"random" + 0.009*"lemma" + 0.008*"network" + 0.007*"iter"
2023-01-09 11:57:04,569 : INFO : topic diff=2.919734, rho=0.353553
2023-01-09 11:57:14,879 : INFO : PROGRESS: pass 0, at document #18000/58520
2023-01-09 11:57:28,437 : INFO : optimized alpha [0.020410713, 0.025178963, 0.020693503, 0.022696337, 0.025431734, 0.030280853, 0.02070847, 0.02338034, 0.02523351, 0.026549114, 0.031427197, 0.0283

...

2023-01-09 15:54:08,633 : INFO : running online (multi-pass) LDA training, 65 topics, 10 passes over the supplied corpus of 58520 documents, updating model once every 2000 documents, evaluating perplexity every 0 documents, iterating 50x with a convergence threshold of 0.001000
2023-01-09 15:54:18,705 : INFO : PROGRESS: pass 0, at document #2000/58520
2023-01-09 15:54:30,520 : INFO : optimized alpha [0.026942555, 0.02666945, 0.026583778, 0.026687872, 0.02644869, 0.027053867, 0.026943063, 0.026650343, 0.027153824, 0.026808934, 0.026553644, 0.027039569, 0.02691847, 0.027050171, 0.026757415, 0.02692039, 0.02689687, 0.027248941, 0.026801907, 0.02694964, 0.027160175, 0.026994895, 0.027066264, 0.02667878, 0.026493039, 0.02721332, 0.026567083, 0.02700369, 0.027311543, 0.02670569, 0.02710982, 0.0264654, 0.027356686, 0.026884563, 0.026544407, 0.026701935, 0.027049415, 0.027303629, 0.026592996, 0.026881887, 0.026855852, 0.027143802, 0.027130427, 0.027054396, 0.026407322, 0.026892532, 0.026946813

2023-01-09 15:55:40,503 : INFO : topic #2 (0.022): 0.019*"fiber" + 0.016*"sound" + 0.014*"frequenc" + 0.014*"hair" + 0.010*"stimul" + 0.008*"bead" + 0.007*"auditori" + 0.007*"cochlear" + 0.007*"stereocilia" + 0.007*"nois"
2023-01-09 15:55:40,507 : INFO : topic #16 (0.030): 0.014*"temperatur" + 0.012*"film" + 0.010*"devic" + 0.009*"layer" + 0.006*"heat" + 0.006*"thermal" + 0.006*"particl" + 0.005*"contact" + 0.005*"substrat" + 0.005*"silicon"
2023-01-09 15:55:40,509 : INFO : topic #4 (0.031): 0.012*"industri" + 0.011*"compani" + 0.009*"market" + 0.009*"servic" + 0.009*"busi" + 0.007*"govern" + 0.006*"econom" + 0.005*"invest" + 0.005*"polici" + 0.005*"firm"
2023-01-09 15:55:40,511 : INFO : topic #25 (0.032): 0.025*"user" + 0.017*"imag" + 0.010*"game" + 0.007*"file" + 0.006*"algorithm" + 0.006*"server" + 0.006*"interfac" + 0.006*"player" + 0.005*"visual" + 0.005*"video"
2023-01-09 15:55:40,522 : INFO : topic diff=2.909546, rho=0.500000
2023-01-09 15:55:51,385 : INFO : PROGRESS: pass 0, at

2023-01-09 15:57:15,566 : INFO : merging changes from 2000 documents into a model of 58520 documents
2023-01-09 15:57:15,818 : INFO : topic #20 (0.019): 0.086*"machin" + 0.046*"cut" + 0.043*"wear" + 0.016*"workpiec" + 0.015*"eros" + 0.014*"scratch" + 0.009*"contact" + 0.009*"runner" + 0.008*"finish" + 0.007*"lubric"
2023-01-09 15:57:15,821 : INFO : topic #23 (0.019): 0.029*"iron" + 0.013*"carbon" + 0.012*"splice" + 0.010*"ocean" + 0.010*"marin" + 0.008*"martensit" + 0.007*"abund" + 0.006*"cell" + 0.006*"cultur" + 0.006*"phytoplankton"
2023-01-09 15:57:15,823 : INFO : topic #13 (0.030): 0.032*"frequenc" + 0.022*"signal" + 0.021*"wave" + 0.015*"puls" + 0.012*"nois" + 0.012*"magnet" + 0.010*"mode" + 0.010*"beam" + 0.009*"amplitud" + 0.008*"probe"
2023-01-09 15:57:15,825 : INFO : topic #4 (0.031): 0.017*"industri" + 0.016*"compani" + 0.015*"busi" + 0.012*"servic" + 0.011*"market" + 0.008*"govern" + 0.008*"econom" + 0.007*"firm" + 0.007*"innov" + 0.006*"countri"
2023-01-09 15:57:15,827 : IN

2023-01-09 15:59:01,400 : INFO : merging changes from 2000 documents into a model of 58520 documents
2023-01-09 15:59:01,663 : INFO : topic #23 (0.018): 0.028*"iron" + 0.016*"carbon" + 0.011*"marin" + 0.009*"ocean" + 0.009*"microbi" + 0.009*"abund" + 0.009*"nutrient" + 0.008*"speci" + 0.008*"growth" + 0.007*"bacteria"
2023-01-09 15:59:01,668 : INFO : topic #1 (0.018): 0.035*"vowel" + 0.030*"tone" + 0.021*"segment" + 0.017*"syllabl" + 0.016*"thruster" + 0.016*"voic" + 0.015*"pitch" + 0.015*"speech" + 0.013*"stress" + 0.013*"conson"
2023-01-09 15:59:01,671 : INFO : topic #39 (0.032): 0.014*"architectur" + 0.007*"cultur" + 0.005*"histori" + 0.005*"centuri" + 0.005*"citi" + 0.004*"modern" + 0.004*"hous" + 0.004*"live" + 0.004*"architect" + 0.003*"museum"
2023-01-09 15:59:01,674 : INFO : topic #4 (0.033): 0.021*"industri" + 0.019*"compani" + 0.015*"busi" + 0.014*"market" + 0.012*"servic" + 0.011*"firm" + 0.010*"govern" + 0.009*"innov" + 0.008*"econom" + 0.007*"countri"
2023-01-09 15:59:01,6

KeyboardInterrupt: 

* 10 topics: 0.6612879085854633 
* 15 topics: 0.6639288994005009
* 20 topics: 0.7000287678468625
* 25 topics:, 0.6915824547608532
* 30 topics: 0.6924143718277599
* 35 topics: 0.701246357671909
* 40 topics: 0.6965632313596052
* 45 topics: 0.6989573216919259
* 50 topics: 0.7013887893321301
* 55 topics: 0.7124166808223836
* 60 topics: 0.701336791986567

Since it has started to plateau, no need to try higher topics. Let's use 55.

In [14]:
lda_model = gensim.models.LdaModel(
        corpus=bow,
        id2word=id2word,
        alpha='auto',
        eta='auto',
        num_topics=55,
        passes=passes,
        eval_every=None
    )

2023-01-09 18:31:48,798 : INFO : using autotuned alpha, starting with [0.018181818, 0.018181818, 0.018181818, 0.018181818, 0.018181818, 0.018181818, 0.018181818, 0.018181818, 0.018181818, 0.018181818, 0.018181818, 0.018181818, 0.018181818, 0.018181818, 0.018181818, 0.018181818, 0.018181818, 0.018181818, 0.018181818, 0.018181818, 0.018181818, 0.018181818, 0.018181818, 0.018181818, 0.018181818, 0.018181818, 0.018181818, 0.018181818, 0.018181818, 0.018181818, 0.018181818, 0.018181818, 0.018181818, 0.018181818, 0.018181818, 0.018181818, 0.018181818, 0.018181818, 0.018181818, 0.018181818, 0.018181818, 0.018181818, 0.018181818, 0.018181818, 0.018181818, 0.018181818, 0.018181818, 0.018181818, 0.018181818, 0.018181818, 0.018181818, 0.018181818, 0.018181818, 0.018181818, 0.018181818]
2023-01-09 18:31:48,815 : INFO : using serial LDA version on this node
2023-01-09 18:31:49,660 : INFO : running online (multi-pass) LDA training, 55 topics, 10 passes over the supplied corpus of 58520 documents, up

2023-01-09 18:33:21,059 : INFO : merging changes from 2000 documents into a model of 58520 documents
2023-01-09 18:33:21,261 : INFO : topic #6 (0.026): 0.026*"coil" + 0.009*"const" + 0.005*"temperatur" + 0.004*"cell" + 0.004*"voltag" + 0.004*"simul" + 0.003*"film" + 0.003*"disp" + 0.003*"layer" + 0.003*"frequenc"
2023-01-09 18:33:21,264 : INFO : topic #2 (0.026): 0.012*"bind" + 0.012*"sentenc" + 0.012*"verb" + 0.010*"claus" + 0.009*"complement" + 0.009*"semant" + 0.008*"phrase" + 0.008*"languag" + 0.007*"pronoun" + 0.007*"argument"
2023-01-09 18:33:21,266 : INFO : topic #41 (0.034): 0.015*"price" + 0.013*"market" + 0.009*"demand" + 0.009*"econom" + 0.009*"polici" + 0.009*"vehicl" + 0.009*"invest" + 0.007*"industri" + 0.007*"decis" + 0.007*"emiss"
2023-01-09 18:33:21,268 : INFO : topic #12 (0.035): 0.022*"algorithm" + 0.011*"matrix" + 0.010*"theorem" + 0.009*"vector" + 0.009*"graph" + 0.009*"network" + 0.008*"proof" + 0.008*"random" + 0.007*"lemma" + 0.006*"edg"
2023-01-09 18:33:21,271 

2023-01-09 18:34:51,048 : INFO : merging changes from 2000 documents into a model of 58520 documents
2023-01-09 18:34:51,259 : INFO : topic #16 (0.021): 0.065*"speech" + 0.023*"recognit" + 0.021*"utter" + 0.018*"speaker" + 0.012*"recogn" + 0.012*"phone" + 0.012*"speak" + 0.009*"acoust" + 0.008*"vocabulari" + 0.008*"morph"
2023-01-09 18:34:51,261 : INFO : topic #2 (0.022): 0.025*"sentenc" + 0.020*"verb" + 0.019*"languag" + 0.015*"claus" + 0.015*"semant" + 0.014*"predic" + 0.013*"argument" + 0.013*"phrase" + 0.012*"grammar" + 0.012*"noun"
2023-01-09 18:34:51,263 : INFO : topic #31 (0.035): 0.031*"imag" + 0.018*"train" + 0.012*"cluster" + 0.011*"algorithm" + 0.011*"network" + 0.010*"dataset" + 0.009*"detect" + 0.007*"segment" + 0.007*"visual" + 0.007*"neural"
2023-01-09 18:34:51,266 : INFO : topic #12 (0.038): 0.024*"algorithm" + 0.012*"matrix" + 0.011*"theorem" + 0.011*"vector" + 0.011*"proof" + 0.010*"graph" + 0.009*"random" + 0.008*"lemma" + 0.007*"iter" + 0.007*"bound"
2023-01-09 18:3

2023-01-09 18:36:25,739 : INFO : merging changes from 2000 documents into a model of 58520 documents
2023-01-09 18:36:25,959 : INFO : topic #16 (0.020): 0.064*"speech" + 0.032*"recognit" + 0.020*"speaker" + 0.020*"acoust" + 0.018*"phone" + 0.018*"recogn" + 0.017*"utter" + 0.014*"speak" + 0.014*"train" + 0.014*"languag"
2023-01-09 18:36:25,961 : INFO : topic #27 (0.020): 0.030*"bone" + 0.029*"joint" + 0.025*"tissu" + 0.020*"collagen" + 0.020*"cartilag" + 0.014*"compress" + 0.012*"load" + 0.011*"ligament" + 0.011*"injuri" + 0.010*"matrix"
2023-01-09 18:36:25,964 : INFO : topic #31 (0.038): 0.035*"imag" + 0.020*"train" + 0.013*"algorithm" + 0.013*"cluster" + 0.011*"network" + 0.011*"dataset" + 0.008*"detect" + 0.007*"neural" + 0.007*"pixel" + 0.007*"visual"
2023-01-09 18:36:25,966 : INFO : topic #44 (0.041): 0.041*"user" + 0.011*"interfac" + 0.009*"game" + 0.008*"file" + 0.008*"display" + 0.006*"particip" + 0.006*"visual" + 0.006*"softwar" + 0.006*"text" + 0.006*"media"
2023-01-09 18:36:2

2023-01-09 18:37:59,941 : INFO : merging changes from 2000 documents into a model of 58520 documents
2023-01-09 18:38:00,163 : INFO : topic #7 (0.019): 0.034*"vowel" + 0.030*"bubbl" + 0.021*"stress" + 0.020*"segment" + 0.020*"tone" + 0.019*"syllabl" + 0.017*"conson" + 0.015*"voic" + 0.015*"stem" + 0.012*"phonolog"
2023-01-09 18:38:00,165 : INFO : topic #27 (0.019): 0.041*"joint" + 0.030*"bone" + 0.025*"cartilag" + 0.023*"tissu" + 0.017*"collagen" + 0.012*"knee" + 0.010*"defect" + 0.010*"compress" + 0.010*"scaffold" + 0.010*"load"
2023-01-09 18:38:00,168 : INFO : topic #31 (0.041): 0.035*"imag" + 0.019*"train" + 0.015*"cluster" + 0.014*"algorithm" + 0.012*"dataset" + 0.010*"network" + 0.008*"pixel" + 0.008*"detect" + 0.007*"segment" + 0.007*"visual"
2023-01-09 18:38:00,170 : INFO : topic #44 (0.044): 0.047*"user" + 0.012*"interfac" + 0.009*"game" + 0.009*"display" + 0.008*"file" + 0.007*"video" + 0.007*"visual" + 0.006*"particip" + 0.006*"text" + 0.006*"softwar"
2023-01-09 18:38:00,172 

...

2023-01-09 20:26:02,467 : INFO : merging changes from 2000 documents into a model of 58520 documents
2023-01-09 20:26:02,681 : INFO : topic #20 (0.016): 0.054*"cell" + 0.022*"tumor" + 0.022*"cancer" + 0.013*"mice" + 0.010*"human" + 0.009*"induc" + 0.009*"receptor" + 0.008*"treatment" + 0.008*"repair" + 0.008*"target"
2023-01-09 20:26:02,683 : INFO : topic #27 (0.018): 0.032*"joint" + 0.026*"muscl" + 0.026*"bone" + 0.024*"tissu" + 0.022*"collagen" + 0.020*"scaffold" + 0.015*"cartilag" + 0.013*"knee" + 0.011*"matrix" + 0.010*"regener"
2023-01-09 20:26:02,685 : INFO : topic #24 (0.089): 0.028*"decis" + 0.024*"risk" + 0.013*"team" + 0.013*"assess" + 0.013*"uncertainti" + 0.013*"safeti" + 0.012*"failur" + 0.012*"scenario" + 0.011*"resourc" + 0.010*"simul"
2023-01-09 20:26:02,687 : INFO : topic #48 (0.099): 0.011*"human" + 0.010*"thing" + 0.009*"say" + 0.007*"agent" + 0.007*"stori" + 0.006*"feel" + 0.006*"kind" + 0.006*"particip" + 0.006*"belief" + 0.006*"social"
2023-01-09 20:26:02,688 : IN

2023-01-09 20:27:43,861 : INFO : merging changes from 2000 documents into a model of 58520 documents
2023-01-09 20:27:44,074 : INFO : topic #20 (0.016): 0.054*"cell" + 0.024*"tumor" + 0.022*"cancer" + 0.013*"mice" + 0.010*"human" + 0.009*"receptor" + 0.009*"induc" + 0.008*"target" + 0.008*"treatment" + 0.008*"signal"
2023-01-09 20:27:44,076 : INFO : topic #27 (0.018): 0.032*"joint" + 0.027*"muscl" + 0.026*"bone" + 0.021*"tissu" + 0.021*"collagen" + 0.019*"cartilag" + 0.016*"knee" + 0.015*"scaffold" + 0.010*"matrix" + 0.010*"load"
2023-01-09 20:27:44,078 : INFO : topic #24 (0.089): 0.029*"decis" + 0.025*"risk" + 0.014*"team" + 0.013*"uncertainti" + 0.013*"failur" + 0.013*"assess" + 0.012*"safeti" + 0.011*"scenario" + 0.010*"resourc" + 0.010*"option"
2023-01-09 20:27:44,080 : INFO : topic #48 (0.099): 0.011*"human" + 0.009*"thing" + 0.008*"say" + 0.007*"agent" + 0.007*"stori" + 0.006*"social" + 0.006*"feel" + 0.006*"particip" + 0.006*"kind" + 0.006*"belief"
2023-01-09 20:27:44,081 : INFO

2023-01-09 20:29:27,079 : INFO : merging changes from 2000 documents into a model of 58520 documents
2023-01-09 20:29:27,293 : INFO : topic #20 (0.016): 0.054*"cell" + 0.025*"tumor" + 0.023*"cancer" + 0.013*"mice" + 0.010*"human" + 0.009*"induc" + 0.008*"target" + 0.008*"treatment" + 0.008*"signal" + 0.008*"receptor"
2023-01-09 20:29:27,295 : INFO : topic #27 (0.018): 0.044*"joint" + 0.027*"muscl" + 0.026*"collagen" + 0.026*"bone" + 0.021*"tissu" + 0.015*"cartilag" + 0.014*"knee" + 0.013*"scaffold" + 0.011*"matrix" + 0.011*"injuri"
2023-01-09 20:29:27,297 : INFO : topic #24 (0.089): 0.029*"decis" + 0.023*"risk" + 0.014*"team" + 0.013*"failur" + 0.013*"uncertainti" + 0.012*"assess" + 0.012*"safeti" + 0.011*"scenario" + 0.010*"option" + 0.010*"simul"
2023-01-09 20:29:27,299 : INFO : topic #48 (0.099): 0.011*"human" + 0.009*"thing" + 0.008*"say" + 0.007*"stori" + 0.007*"agent" + 0.006*"belief" + 0.006*"kind" + 0.006*"social" + 0.006*"feel" + 0.006*"self"
2023-01-09 20:29:27,301 : INFO : t

2023-01-09 20:31:07,610 : INFO : merging changes from 2000 documents into a model of 58520 documents
2023-01-09 20:31:07,825 : INFO : topic #20 (0.016): 0.054*"cell" + 0.023*"tumor" + 0.022*"cancer" + 0.012*"mice" + 0.010*"human" + 0.009*"induc" + 0.008*"target" + 0.008*"receptor" + 0.008*"treatment" + 0.008*"signal"
2023-01-09 20:31:07,827 : INFO : topic #27 (0.018): 0.041*"joint" + 0.024*"muscl" + 0.023*"collagen" + 0.023*"bone" + 0.021*"tissu" + 0.018*"cartilag" + 0.014*"knee" + 0.013*"scaffold" + 0.013*"injuri" + 0.013*"regener"
2023-01-09 20:31:07,828 : INFO : topic #24 (0.089): 0.029*"decis" + 0.023*"risk" + 0.015*"team" + 0.013*"safeti" + 0.013*"assess" + 0.013*"uncertainti" + 0.013*"failur" + 0.011*"simul" + 0.011*"option" + 0.011*"scenario"
2023-01-09 20:31:07,830 : INFO : topic #48 (0.100): 0.011*"human" + 0.010*"thing" + 0.008*"say" + 0.007*"agent" + 0.006*"stori" + 0.006*"social" + 0.006*"kind" + 0.006*"belief" + 0.006*"self" + 0.006*"particip"
2023-01-09 20:31:07,832 : INF

2023-01-09 20:32:30,293 : INFO : merging changes from 520 documents into a model of 58520 documents
2023-01-09 20:32:30,508 : INFO : topic #20 (0.016): 0.056*"cell" + 0.023*"tumor" + 0.022*"cancer" + 0.011*"mice" + 0.010*"human" + 0.009*"repair" + 0.008*"induc" + 0.008*"target" + 0.008*"receptor" + 0.008*"treatment"
2023-01-09 20:32:30,509 : INFO : topic #27 (0.018): 0.032*"joint" + 0.027*"bone" + 0.023*"muscl" + 0.023*"tissu" + 0.021*"collagen" + 0.020*"cartilag" + 0.013*"matrix" + 0.012*"knee" + 0.011*"regener" + 0.010*"defect"
2023-01-09 20:32:30,511 : INFO : topic #24 (0.090): 0.028*"decis" + 0.021*"risk" + 0.015*"team" + 0.014*"assess" + 0.013*"uncertainti" + 0.013*"failur" + 0.012*"resourc" + 0.012*"scenario" + 0.011*"safeti" + 0.011*"simul"
2023-01-09 20:32:30,513 : INFO : topic #48 (0.101): 0.011*"human" + 0.010*"thing" + 0.008*"say" + 0.008*"stori" + 0.006*"belief" + 0.006*"self" + 0.006*"agent" + 0.006*"particip" + 0.006*"feel" + 0.006*"kind"
2023-01-09 20:32:30,515 : INFO : 

In [18]:
from gensim.test.utils import datapath
file = datapath("model")
lda_model.save(file)
# lda = LdaModel.load(temp_file)

2023-01-09 21:04:45,340 : INFO : LdaState lifecycle event {'fname_or_handle': '/usr/local/lib/python3.10/site-packages/gensim/test/test_data/model.state', 'separately': 'None', 'sep_limit': 10485760, 'ignore': frozenset(), 'datetime': '2023-01-09T21:04:45.340516', 'gensim': '4.3.0', 'python': '3.10.6 (main, Aug 30 2022, 05:12:36) [Clang 13.1.6 (clang-1316.0.21.2.5)]', 'platform': 'macOS-12.5.1-x86_64-i386-64bit', 'event': 'saving'}
2023-01-09 21:04:45,362 : INFO : saved /usr/local/lib/python3.10/site-packages/gensim/test/test_data/model.state
2023-01-09 21:04:45,396 : INFO : LdaModel lifecycle event {'fname_or_handle': '/usr/local/lib/python3.10/site-packages/gensim/test/test_data/model', 'separately': "['expElogbeta', 'sstats']", 'sep_limit': 10485760, 'ignore': ['dispatcher', 'id2word', 'state'], 'datetime': '2023-01-09T21:04:45.396080', 'gensim': '4.3.0', 'python': '3.10.6 (main, Aug 30 2022, 05:12:36) [Clang 13.1.6 (clang-1316.0.21.2.5)]', 'platform': 'macOS-12.5.1-x86_64-i386-64bi

## Evaluation

In [23]:
import pyLDAvis.gensim_models as gensimvis
import pickle 
import pyLDAvis

# Visualize the topics
pyLDAvis.enable_notebook()

LDAvis_data_filepath = os.path.join('./ldavis')

# # this is a bit time consuming - make the if statement True
# # if you want to execute visualization prep yourself
if 1 == 1:
    LDAvis_prepared = gensimvis.prepare(lda_model, bow, dictionary)
    with open(LDAvis_data_filepath, 'wb') as f:
        pickle.dump(LDAvis_prepared, f)

# load the pre-prepared pyLDAvis data from disk
with open(LDAvis_data_filepath, 'rb') as f:
    LDAvis_prepared = pickle.load(f)

pyLDAvis.save_html(LDAvis_prepared, './ldavis.html')

LDAvis_prepared

/usr/local/lib/python3.10/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib and slated for removal in Python 3.12; see the module's documentation for alternative uses
  from imp import reload
/usr/local/lib/python3.10/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib and slated for removal in Python 3.12; see the module's documentation for alternative uses
  from imp import reload
/usr/local/lib/python3.10/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib and slated for removal in Python 3.12; see the module's documentation for alternative uses
  from imp import reload
/usr/local/lib/python3.10/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib and slated for removal in Python 3.12; see the module's documentation for alternative uses
  from im

PreparedData(topic_coordinates=              x         y  topics  cluster      Freq
topic                                               
53     0.178863 -0.091694       1        1  3.611730
12    -0.033668  0.181987       2        1  3.469817
30     0.245330 -0.070884       3        1  3.304028
10     0.136642 -0.066305       4        1  3.068550
22     0.217017 -0.043649       5        1  2.851511
48     0.152202 -0.004022       6        1  2.830490
31    -0.024508  0.127976       7        1  2.780238
17    -0.107737 -0.169582       8        1  2.722764
33    -0.046009 -0.153208       9        1  2.513302
19     0.218487 -0.075629      10        1  2.507901
24     0.137409  0.061826      11        1  2.377955
1     -0.132639 -0.138056      12        1  2.308713
5     -0.195623  0.091608      13        1  2.299068
37    -0.178828 -0.064113      14        1  2.242770
52    -0.158837  0.026241      15        1  2.096869
36    -0.183052  0.075857      16        1  2.092867
28     0.194621 -0.080452      17        1  2.034151
42    -0.122845  0.007144      18        1  2.010485
45    -0.095364  0.130485      19        1  1.994257
40    -0.166006 -0.149818      20        1  1.978380
34     0.120234  0.009830      21        1  1.976037
18    -0.165493  0.036223      22        1  1.961980
43    -0.189830 -0.065139      23        1  1.942128
44     0.098526  0.087830      24        1  1.877515
3      0.018089 -0.162937      25        1  1.839191
8      0.208892 -0.093382      26        1  1.832589
47    -0.097686  0.017363      27        1  1.798258
32    -0.098194  0.149546      28        1  1.780448
46     0.056064  0.035659      29        1  1.764500
23    -0.087648 -0.076243      30        1  1.681515
41     0.105513 -0.068026      31        1  1.667074
29     0.102893  0.188977      32        1  1.664243
9      0.118764  0.034945      33        1  1.622278
54     0.199674 -0.046112      34        1  1.609298
20    -0.041784 -0.228554      35        1  1.602105
4      0.147992  0.036845      36        1  1.596394
39     0.000285  0.099064      37        1  1.577043
49    -0.105634  0.126858      38        1  1.553227
50    -0.138064 -0.018087      39        1  1.492864
51    -0.121992 -0.176191      40        1  1.463869
21     0.064676  0.153883      41        1  1.396846
11     0.007019  0.012977      42        1  1.375917
26     0.034429  0.177352      43        1  1.265191
35    -0.081872  0.033583      44        1  1.117800
2      0.087447  0.055297      45        1  1.111967
13    -0.158858  0.066953      46        1  1.070957
16     0.075971  0.131294      47        1  1.030109
25     0.114711 -0.124320      48        1  0.993692
0     -0.039836 -0.101323      49        1  0.944143
6     -0.010730  0.184103      50        1  0.933428
38    -0.104806 -0.141725      51        1  0.829947
14    -0.191296  0.048018      52        1  0.815238
15     0.087840  0.136337      53        1  0.684055
27    -0.075277 -0.162038      54        1  0.521203
7      0.024525  0.045426      55        1  0.511107, topic_info=            Term          Freq         Total Category  logprob  loglift
2198        cell  1.579891e+06  1.579891e+06  Default  30.0000  30.0000
2080  temperatur  1.069703e+06  1.069703e+06  Default  29.0000  29.0000
903      network  8.069260e+05  8.069260e+05  Default  28.0000  28.0000
1456       water  9.141010e+05  9.141010e+05  Default  27.0000  27.0000
1758        imag  9.039880e+05  9.039880e+05  Default  26.0000  26.0000
...          ...           ...           ...      ...      ...      ...
767      languag  1.521791e+04  2.502025e+05  Topic55  -4.7645   2.4765
4143        root  1.253161e+04  1.450785e+05  Topic55  -4.9587   2.8273
154     boundari  1.352726e+04  4.555416e+05  Topic55  -4.8822   1.7596
232      cluster  1.220142e+04  3.007488e+05  Topic55  -4.9854   2.0716
1991     sequenc  1.087018e+04  4.908101e+05  Topic55  -5.1009   1.4663

[4746 rows x 6 columns], token_table=       Topic      Freq         Ter

In [43]:
columns = ["filename"]
for i in range(55): columns.append("topic_" + str(i))
df = pd.DataFrame(columns=columns)

In [44]:
files = [x for x in os.listdir("./full_text_processed") if ".pdf.txt" in x]

for fname, doc in zip(files, bow):
    row = {"filename": fname}
    for topic, probability in lda_model.get_document_topics(doc, minimum_probability=0):
       row["topic_" + str(topic)] = probability
    df = df.append(row, ignore_index=True)

print(df.shape)
df.head()

(58520, 56)


,filename,topic_0,topic_1,topic_2,topic_3,topic_4,topic_5,topic_6,topic_7,topic_8,...,topic_45,topic_46,topic_47,topic_48,topic_49,topic_50,topic_51,topic_52,topic_53,topic_54
0,759082635-MIT.pdf.txt,0.000003,0.000004,0.000003,0.004166,0.000008,0.000007,0.000278,0.000003,0.596040,...,0.000008,0.000019,0.000010,0.076461,0.000007,0.000007,0.000181,0.000005,0.041731,0.042416
1,1191837094-MIT.pdf.txt,0.011932,0.000004,0.000003,0.000007,0.008315,0.011860,0.006571,0.000003,0.007055,...,0.271774,0.000021,0.093987,0.017976,0.006622,0.000007,0.000006,0.000006,0.000009,0.052831
2,41567358-MIT.pdf.txt,0.021394,0.000009,0.000007,0.000015,0.000019,0.000018,0.152327,0.069201,0.001521,...,0.003529,0.014839,0.000025,0.000036,0.000017,0.000016,0.000014,0.000012,0.000021,0.014326
3,54809434-MIT.pdf.txt,0.008601,0.009254,0.007718,0.016503,0.020696,0.018735,0.018735,0.007555,0.019830,...,0.021747,0.049072,0.026556,0.038071,0.018587,0.017210,0.015158,0.013189,0.022066,0.028531
4,858871740-MIT.pdf.txt,0.000009,0.000009,0.000008,0.000017,0.000021,0.000019,0.174587,0.055833,0.000020,...,0.078201,0.010431,0.000027,0.000039,0.000019,0.000018,0.000015,0.000013,0.011102,0.006820


In [45]:
df.to_csv("embeddings.csv")

## Project to 2D

In [56]:
from sklearn.manifold import TSNE
tsne = TSNE(n_components=2)
tsne_results = tsne.fit_transform(df.copy().drop("filename", axis=1).values)

In [67]:
tsne_df = pd.DataFrame(tsne_results, columns = ["tsne_0", "tsne_1"])
tsne_df.head()

,tsne_0,tsne_1
0,-53.347881,49.898830
1,13.855901,-7.193710
2,37.740334,48.299267
3,-7.037585,13.036370
4,70.324028,43.869823


In [68]:
projected_df = pd.concat([df, tsne_df], axis=1)
projected_df.head()

,filename,topic_0,topic_1,topic_2,topic_3,topic_4,topic_5,topic_6,topic_7,topic_8,...,topic_47,topic_48,topic_49,topic_50,topic_51,topic_52,topic_53,topic_54,tsne_0,tsne_1
0,759082635-MIT.pdf.txt,0.000003,0.000004,0.000003,0.004166,0.000008,0.000007,0.000278,0.000003,0.596040,...,0.000010,0.076461,0.000007,0.000007,0.000181,0.000005,0.041731,0.042416,-53.347881,49.898830
1,1191837094-MIT.pdf.txt,0.011932,0.000004,0.000003,0.000007,0.008315,0.011860,0.006571,0.000003,0.007055,...,0.093987,0.017976,0.006622,0.000007,0.000006,0.000006,0.000009,0.052831,13.855901,-7.193710
2,41567358-MIT.pdf.txt,0.021394,0.000009,0.000007,0.000015,0.000019,0.000018,0.152327,0.069201,0.001521,...,0.000025,0.000036,0.000017,0.000016,0.000014,0.000012,0.000021,0.014326,37.740334,48.299267
3,54809434-MIT.pdf.txt,0.008601,0.009254,0.007718,0.016503,0.020696,0.018735,0.018735,0.007555,0.019830,...,0.026556,0.038071,0.018587,0.017210,0.015158,0.013189,0.022066,0.028531,-7.037585,13.036370
4,858871740-MIT.pdf.txt,0.000009,0.000009,0.000008,0.000017,0.000021,0.000019,0.174587,0.055833,0.000020,...,0.000027,0.000039,0.000019,0.000018,0.000015,0.000013,0.011102,0.006820,70.324028,43.869823


In [69]:
projected_df.to_csv("projected.csv")

Let's merge with metadata and save it

In [ ]:
metadata = pd.read_csv("metadata.csv")
print(metadata.shape)
metadata.head()

In [71]:
merged = projected_df.merge(metadata, how='inner', on="filename")


In [79]:
merged = merged.drop('Unnamed: 0', axis=1)

In [80]:
print(merged.shape)
merged.head()

(57763, 69)


,filename,topic_0,topic_1,topic_2,topic_3,topic_4,topic_5,topic_6,topic_7,topic_8,...,department,issued,uri,oclc,abstract,subject,title,image_url,handle,identifier
0,759082635-MIT.pdf.txt,0.000003,0.000004,0.000003,0.004166,0.000008,0.000007,0.000278,0.000003,0.596040,...,Massachusetts Institute of Technology. Dept. o...,2011,http://hdl.handle.net/1721.1/67217,7.590826e+08,"Personal teacher self-efficacy (PTE), or the b...",Urban Studies and Planning.,Teacher self-efficacy in Cape Town : a bottom ...,https://mit.atmire.com/bitstream/1721.1/67217/...,1721.1/67217,oai:mit.atmire.com:1721.1/67217
1,1191837094-MIT.pdf.txt,0.011932,0.000004,0.000003,0.000007,0.008315,0.011860,0.006571,0.000003,0.007055,...,Massachusetts Institute of Technology. Departm...,2020,https://hdl.handle.net/1721.1/127125,1.191837e+09,"In the age of automation and computation, smar...",Mechanical Engineering.,Design of dynamically controlled desktop fiber...,https://dspace.mit.edu/bitstream/1721.1/127125...,1721.1/127125,oai:dspace.mit.edu:1721.1/127125
2,41567358-MIT.pdf.txt,0.021394,0.000009,0.000007,0.000015,0.000019,0.000018,0.152327,0.069201,0.001521,...,Massachusetts Institute of Technology. Dept. o...,1998,http://hdl.handle.net/1721.1/46227,4.156736e+07,NaN,Electrical Engineering and Computer Science,A study of improvements for the Winbank Courte...,https://mit.atmire.com/bitstream/1721.1/46227/...,1721.1/46227,oai:mit.atmire.com:1721.1/46227
3,54809434-MIT.pdf.txt,0.008601,0.009254,0.007718,0.016503,0.020696,0.018735,0.018735,0.007555,0.019830,...,Massachusetts Institute of Technology. Dept. o...,2003,http://hdl.handle.net/1721.1/7976,5.480943e+07,Microchips for constant release are not a new ...,Materials Science and Engineering.,Controlled release microchip,https://mit.atmire.com/bitstream/1721.1/7976/5...,1721.1/7976,oai:mit.atmire.com:1721.1/7976
4,858871740-MIT.pdf.txt,0.000009,0.000009,0.000008,0.000017,0.000021,0.000019,0.174587,0.055833,0.000020,...,Massachusetts Institute of Technology. Departm...,2013,http://hdl.handle.net/1721.1/81609,8.588717e+08,One of the biggest challenges in implementing ...,Mechanical Engineering.,Building blocks for co-design of controllers a...,https://mit.atmire.com/bitstream/1721.1/81609/...,1721.1/81609,oai:mit.atmire.com:1721.1/81609


In [84]:
merged = merged.drop_duplicates(subset=['filename'])

In [85]:
merged.to_csv("merged.csv", index = False)